# Functions and imports
(Incluir no artigo)
Although AI-Feynman is an extremely effective algorithm in terms of error metrics, it may fail to identify certain structures due to its internal automation, which applies multiple processes and transformations to the data and can result in overly complex expressions. It is also a relatively slow algorithm; however, it allows symbolic search escalation to be interrupted at any iteration, reducing the time needed for scientists to obtain a useful expression.

Furthermore, the algorithm is prone to failure when it cannot identify an adequate underlying dynamic, as its search process is guided by the Minimum Description Length principle, which aims to balance error minimization with symbolic simplification of the discovered expression.

Another relevant aspect is AI-Feynman's difficulty in reliably reproducing the obtained expressions. This stems from the stochastic nature of its evolutionary process, which uses symbolic representations in Reverse Polish Notation—a method that favors computational efficiency but also contributes to result variability and, in many cases, makes it impossible to recover the original structural form that generated the data.




**Installing dependencies**

*Run these cells if this is the first time that this notebook is being executed.*

In [ ]:
#!pip install aifeynman

In [ ]:
# "Whenever the kernel is restarted or interrupted, re-run the cell that
# imports the packages and defines the error_eval function.

import numpy as np
from scipy.integrate import solve_ivp, odeint
import pandas as pd
import itertools
import matplotlib.pyplot as plt

import random
random.seed(100)
np.random.seed(100)

#Error eval
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from scipy import stats



## Definitions and data

All structures below are quite similar, but we need to have a cell for each dynamics to take the equations into consideration correctly.

In [ ]:
def volterra(t, y, a=2, b=0.5, c=1, d=0.375):
  u,v = y
  dudt = a*u - b*u*v
  dvdt = -c*v + d*b*u*v

  return [dudt, dvdt]

#Parameters
feature_names = ['u', 'v']

t_start = 0.0
t_end = 7.5
t_step = 0.1
t_eval = np.arange(t_start, t_end, t_step)
t_span = (t_start, t_end)

state0 = np.array([20, 5])

#Solve the system
solution = solve_ivp(volterra, t_span, state0, t_eval=t_eval)
u, v = solution.y

#Target variables
dudt = volterra([0], solution.y)[0]
dvdt = volterra([0], solution.y)[1]
targets = [dudt,dvdt]

du = np.column_stack((solution.y.T, dudt))
np.savetxt('dados_LotkU.txt', pd.DataFrame(du))

dv = np.column_stack((solution.y.T, dvdt))
np.savetxt('dados_LotkV.txt', pd.DataFrame(dv))

data = np.column_stack(( u, v, dudt, dvdt))
np.savetxt('LotkV_solution.txt', data, header='u v dudt dvdt', comments='')

In [ ]:
def pendulum(t, y):
    theta, omega = y
    g = 9.8
    L = 1.0

    d_theta = omega
    d_omega = - (g / L) * np.sin(theta)

    return [d_theta, d_omega]

#Parameters
feature_names = ['theta', 'omega']

t_start = 0.0
t_end = 10
t_step = 0.01
t_eval = np.arange(t_start, t_end, t_step)
t_span = (t_start, t_end)

state0 = np.array([np.radians(45), 0])

#Solve the system
solution = solve_ivp(pendulum, t_span, state0, t_eval=t_eval)
T, O=solution.y
#Target variables
dthetadt = solution.y[1]  # omega diretamente
domegadt = -9.8 * np.sin(solution.y[0])  # -g/L * sin(theta)

targets = [dthetadt, domegadt]

# Salvar os dados em um arquivo CSV
dtheta = np.column_stack((solution.y.T[:, [1]], dthetadt))
np.savetxt('dados_pendT.txt', pd.DataFrame(dtheta))

domega = np.column_stack((solution.y.T[:, [0]], domegadt))
np.savetxt('dados_pendO.txt', pd.DataFrame(domega))

data = np.column_stack(( T, O, dthetadt, domegadt))
np.savetxt('Pendulum_solution.txt', data, header='T O  dthetadt domegadt', comments='')

In [ ]:
def lorenz(t, ivp_sol, sigma=2.0, rho=1.0, beta=2.6):
  x, y, z = ivp_sol
  dx_dt = sigma * (y - x)
  dy_dt = x * (rho - z) - y
  dz_dt = x * y - beta * z

  return [dx_dt, dy_dt, dz_dt]

#Parameters
feature_names = ['X', 'Y', 'Z']
t_start = 0.0
t_end = 5.0
t_step = 0.02
t_eval = np.arange(t_start, t_end, t_step)
t_span = (t_start, t_end)

x0, y0, z0 = 0.6, 2.0, 1.0  # Condições iniciais
state0 = [x0, y0, z0]

#Solve the system
solution = solve_ivp(lorenz, t_span, state0, method='RK45', t_eval=t_eval)
x, y, z=solution.y
#Target variables
dxdt = lorenz([0], solution.y)[0]
dydt = lorenz([0], solution.y)[1]
dzdt = lorenz([0], solution.y)[2]
targets = [dxdt,dydt,dzdt]

# Salvar os dados em um arquivo txt
dx = np.column_stack((solution.y.T[:, [0,1]], dxdt))
np.savetxt('dados_lorenzX.txt', pd.DataFrame(dx))

dy = np.column_stack((solution.y.T, dydt))
np.savetxt('dados_lorenzY.txt', pd.DataFrame(dy))

dz = np.column_stack((solution.y.T, dzdt))
np.savetxt('dados_lorenzZ.txt', pd.DataFrame(dz))

data = np.column_stack(( x, y, z, dxdt, dydt, dzdt))
np.savetxt('Lorenz_solution.txt',data, header='x y z dxdt dydt dzdt', comments='')

In [ ]:
#=== SIS ===
def sis(t, y, beta = 0.3, gamma = 0.1):
  S, I = y
  dSdt = -beta * S * I + gamma * I
  dIdt = beta * S * I - gamma * I
  return [dSdt, dIdt]

#Parameters
feature_names = ['S', 'I']
#Time
t = np.linspace(0, 10, 100)
t_start = 0.0
t_end = 100.0
t_step =  0.1
t_span = (t_start, t_end)
t_eval = np.arange(t_start, t_end, t_step)

#SIS
N = 1
I0 = 0.001
S0 = N - I0
x0 = np.array([S0, I0])
#Solve the SIS system
solution = solve_ivp(sis, t_span, x0, t_eval=t_eval)
S, I=solution.y
#Target variables
dSdt = sis([0], solution.y)[0]
dIdt = sis([0], solution.y)[1]
targets = [dSdt,dIdt]

dS = np.column_stack((solution.y.T[:, [0,1]], dSdt))
np.savetxt('dados_sisS.txt', pd.DataFrame(dS))

dI = np.column_stack((solution.y.T[:, [0,1]], dIdt))
np.savetxt('dados_sisI.txt', pd.DataFrame(dI))

data = np.column_stack(( S, I, dSdt, dIdt))
np.savetxt('SIS_solution.txt', data, header='S I dSdt dIdt', comments='')

In [ ]:
#=== SIR ===
def sir(t, y, beta=0.3, gamma=0.1):
  S, I, R = y
  dSdt = -beta * S * I
  dIdt = beta * S * I - gamma * I
  dRdt = gamma*I
  return [dSdt, dIdt, dRdt]

#Parameters
feature_names = ['S', 'I', 'R']
#Time
t_start = 0.0
t_end = 100.0
t_step =  0.1
t_span = (t_start, t_end)
t_eval = np.arange(t_start, t_end, t_step)

N = 1
I0, R0 = 0.001, 0.0
S0 = N - I0 - R0
x0 = np.array([S0, I0, R0])

#They're hardcoded in the SIR function definition. Just stating them here for reference
beta=0.3
gamma=0.1

#Solve the SIR system
solution = solve_ivp(sir, t_span, x0, t_eval=t_eval)
S, I, R=solution.y
#Target variables
dSdt = sir([0], solution.y)[0]
dIdt = sir([0], solution.y)[1]
dRdt = sir([0], solution.y)[2]
targets = [dSdt, dIdt, dRdt]

dS = np.column_stack((solution.y.T[:, [0,1]], dSdt))
np.savetxt('dados_sirS.txt', pd.DataFrame(dS))

dI = np.column_stack((solution.y.T[:, [0,1]], dIdt))
np.savetxt('dados_sirI.txt', pd.DataFrame(dI))

dR = np.column_stack((solution.y.T[:, [1]], dRdt))
np.savetxt('dados_sirR.txt', pd.DataFrame(dR))

data = np.column_stack((S, I, R, dSdt, dIdt, dRdt))
np.savetxt('SIR_solution.txt', data, header='S I R dSdt dIdt dRdt', comments='')

In [ ]:
#=== SEIR ===
def seir(t, y, beta=0.3, alpha=0.2, gamma=1.0):
    S, E, I, R = y
    dSdt = -beta * S * I
    dEdt = beta * S * I - alpha * E
    dIdt = alpha * E - gamma * I
    dRdt = gamma * I
    return [dSdt, dEdt, dIdt, dRdt]

#Parameters
feature_names = ['S', 'E', 'I', 'R']
#Time
t_start = 0.0
t_end = 100.0
t_step =  0.1   #why does it changes here?
t_span = (t_start, t_end)
t_eval = np.arange(t_start, t_end, t_step)

N = 1
I0, R0, E0 = 0.001, 0.0, 0.0
S0 = N - I0 - R0 - E0
x0 = np.array([S0, E0, I0, R0])

#They're hardcoded in the SIR function definition. Just stating them here for reference
beta = 0.3
gamma = 1.0
alpha = 0.2

#Solve the SEIR system
solution = solve_ivp(seir, t_span, x0, t_eval=t_eval, args=(0.3, 0.2, 1.0))
S, E, I, R=solution.y
#Target variables
dSdt = seir([0], solution.y)[0]
dEdt = seir([0], solution.y)[1]
dIdt = seir([0], solution.y)[2]
dRdt = seir([0], solution.y)[3]
targets = [dSdt,dEdt,dIdt,dRdt]

dS = np.column_stack((solution.y.T[:, [0,2]], dSdt))
np.savetxt('dados_seirS.txt', pd.DataFrame(dS))

dE = np.column_stack((solution.y.T[:, [0,1,2]], dEdt))
np.savetxt('dados_seirE.txt', pd.DataFrame(dE))

dI = np.column_stack((solution.y.T[:, [1, 2]], dIdt))
np.savetxt('dados_seirI.txt', pd.DataFrame(dI))

dR = np.column_stack((solution.y.T[:, [2]], dRdt))
np.savetxt('dados_seirR.txt', pd.DataFrame(dR))

data = np.column_stack((S, E, I, R,dSdt, dEdt, dIdt, dRdt))
np.savetxt('SEIR_solution.txt', data, header='S E I R dSdt dEdt dIdt dRdt', comments='')

In [ ]:
 #=== SEIRD ===
def seird(t, y, beta=0.3, alpha=0.2, gamma=1.0, mu=0.1):
    S, E, I, R, D = y
    dSdt = -beta * S * I
    dEdt = beta * S * I - alpha * E
    dIdt = alpha * E - (gamma + mu) * I
    dRdt = gamma * I
    dDdt = mu * I
    return [dSdt, dEdt, dIdt, dRdt, dDdt]

#Parameters
feature_names = ['S', 'E', 'I', 'R', 'D']
#Time
t_start = 0.0
t_end = 100.0
t_step =  0.05   #why does it changes here?
t_span = (t_start, t_end)
t_eval = np.arange(t_start, t_end, t_step)

N = 1
I0, R0, E0, D0 = 0.001, 0.0, 0.0, 0.0
S0 = N - I0 - R0 - E0 - D0
x0 = np.array([S0, E0, I0, R0, D0])

#They're hardcoded in the SIR function definition. Just stating them here for reference
beta = 0.3
gamma = 1.0
alpha = 0.2
mu = 0.1

#Solve the SEIR system
solution = solve_ivp(seird, t_span, x0, t_eval=t_eval, args=(0.3, 0.2, 1.0, 0.1))
S, E, I, R, D=solution.y
#Target variables
dSdt = seird([0], solution.y)[0]
dEdt = seird([0], solution.y)[1]
dIdt = seird([0], solution.y)[2]
dRdt = seird([0], solution.y)[3]
dDdt = seird([0], solution.y)[4]
targets = [dSdt,dEdt,dIdt,dRdt,dDdt]

dS = np.column_stack((solution.y.T[:, [0,2]], dSdt))
np.savetxt('dados_seirdS.txt', pd.DataFrame(dS))

dE = np.column_stack((solution.y.T[:, [0,1,2]], dEdt))
np.savetxt('dados_seirdE.txt', pd.DataFrame(dE))

dI = np.column_stack((solution.y.T[:, [1, 2]], dIdt))
np.savetxt('dados_seirdI.txt', pd.DataFrame(dI))

dR = np.column_stack((solution.y.T[:, [2]], dRdt))
np.savetxt('dados_seirdR.txt', pd.DataFrame(dR))

dD = np.column_stack((solution.y.T[:, [1]], dDdt))
np.savetxt('dados_seirdD.txt', pd.DataFrame(dD))

data = np.column_stack((S, E, I, R,D, dSdt, dEdt, dIdt, dRdt, dDdt))
np.savetxt('SEIRD_solution.txt', data, header='S E I R D dSdt dEdt dIdt dRdt dDdt', comments='')

In [ ]:
 #=== SIRV ===
def sirv(t, y, beta=0.5, gamma=1.0, nu=0.5):
    S, I, R, V = y
    dSdt = -beta * S * I - nu * S
    dIdt = beta * S * I - gamma * I
    dRdt = gamma * I
    dVdt = nu * S
    return [dSdt, dIdt, dRdt, dVdt]

#Parameters
feature_names = ['S', 'I', 'R', 'V']
#Time
t_start = 0.0
t_end = 100.0
t_step =  0.1
t_span = (t_start, t_end)
t_eval = np.arange(t_start, t_end, t_step)

N = 1
I0, R0, V0 = 0.001, 0.0, 0.5
S0 = N - I0 - R0 - V0
x0 = np.array([S0, I0, R0, V0])

#They're hardcoded in the SIR function definition. Just stating them here for reference
beta=0.3
gamma=0.1
nu=0.5

#Solve the SIR system
solution = solve_ivp(sirv, t_span, x0, t_eval=t_eval)
S, I, R, V = solution.y
#Target variables
dSdt = sirv([0], solution.y)[0]
dIdt = sirv([0], solution.y)[1]
dRdt = sirv([0], solution.y)[2]
dVdt = sirv([0], solution.y)[3]
targets = [dSdt,dIdt,dRdt,dVdt]

dS = np.column_stack((solution.y.T[:, [0,1]], dSdt))
np.savetxt('dados_sirvS.txt', pd.DataFrame(dS))

dI = np.column_stack((solution.y.T[:, [0, 1]], dIdt))
np.savetxt('dados_sirvI.txt', pd.DataFrame(dI))

dR = np.column_stack((solution.y.T[:, [1]], dRdt))
np.savetxt('dados_sirvR.txt', pd.DataFrame(dR))

dV = np.column_stack((solution.y.T[:, [0]], dVdt))
np.savetxt('dados_sirvV.txt', pd.DataFrame(dV))

data = np.column_stack((S, I, R, V, dSdt, dIdt, dRdt, dVdt))
np.savetxt('SIRV_solution.txt',data, header='S I R V dSdt dIdt dRdt dVdt', comments='')

In [ ]:
 #=== SIRS ===
def sirs(t, y, beta=0.3, gamma=1.0, nu=0.2):
    S, I, R = y
    dSdt = -beta * S * I + nu * R
    dIdt =  beta * S * I - gamma * I
    dRdt = gamma * I- nu * R
    return [dSdt, dIdt, dRdt]

#Parameters
feature_names = ['S', 'I', 'R']
#Time
t_start = 0.0
t_end = 100.0
t_step =  0.1
t_span = (t_start, t_end)
t_eval = np.arange(t_start, t_end, t_step)

N = 1
I0, R0 = 0.001, 0.0
S0 = N - I0 - R0
x0 = np.array([S0, I0, R0])

#They're hardcoded in the SIR function definition. Just stating them here for reference
beta=0.3
gamma=0.1

#Solve the SIR system
solution = solve_ivp(sirs, t_span, x0, t_eval=t_eval)
S, I, R = solution.y
#Target variables
dSdt = sirs([0], solution.y)[0]
dIdt = sirs([0], solution.y)[1]
dRdt = sirs([0], solution.y)[2]
targets = [dSdt,dIdt,dRdt]

dS = np.column_stack((solution.y.T[:, [0,1,2]], dSdt))
np.savetxt('dados_sirsS.txt', pd.DataFrame(dS))

dI = np.column_stack((solution.y.T[:, [0, 1]], dIdt))
df = pd.DataFrame(dI, columns=['S', 'I', 'dIdt'])
#np.savetxt('dados_sirsI.txt', pd.DataFrame(dI))
df.to_csv("dados_sirsI.txt", sep="\t", index=False, header=False)



dR = np.column_stack((solution.y.T[:, [1,2]], dRdt))
np.savetxt('dados_sirsR.txt', pd.DataFrame(dR))

data = np.column_stack((S, I, R,dSdt, dIdt, dRdt))
np.savetxt('SIRS_solution.txt', data, header='S I R dSdt dIdt dRdt', comments='')

# Error evaluation

In [ ]:
def error_eval(dyn_name, curves_names, y_preds, targets):
    '''
    This function assumes you've run one of the eval_models() functions previously
    to estimate y_true and y_pred to keep it compatible with all dynamics.
    '''
    mse = []
    mae = []
    r2 = []
    rmse = []
    statistics = []
    p_values = []
    wilcoxon_sol = []

    for i in range(len(y_preds)):
        y_true = targets[i]
        y_pred = y_preds[i]

        mse.append(mean_squared_error(y_true, y_pred))
        mae.append(mean_absolute_error(y_true, y_pred))
        r2.append(r2_score(y_true, y_pred))
        rmse.append(np.sqrt(mean_squared_error(y_true, y_pred)))
        tol = 1e-8  # tolerância para igualdade numérica

        #diffs = np.abs(y_true - y_pred)
        if np.allclose(y_true, y_pred, rtol=1e-5, atol=1e-8):
            stat, p_value = 0.0, 1.0
        else:
            try:
                stat, p_value = stats.wilcoxon(y_true, y_pred, zero_method="pratt")# stats.wilcoxon(y_true, y_pred)
            except ValueError:
                stat, p_value = np.nan, 1.0

        statistics.append(stat)
        p_values.append(p_value)
        wilcoxon_sol.append(1 if p_value < 0.05 else 0)

    res = pd.DataFrame(data={
        'dynamics': [dyn_name]*len(y_preds),
        'axis': curves_names,
        'mae': mae,
        'mse': mse,
        'r2': r2,
        'rmse': rmse,
        'wilcoxon_raw_value': statistics,
        'p-value': p_values,
        'is_difference': wilcoxon_sol
    })

    return res


# Dynamics to investigate

Applies AI-Feynman to the dynamics
Since our approach is supervised, it is based on the assumption that we have prior knowledge of the dynamics of the systems under analysis. Each expression that constitutes the system uses only the essential variables needed to describe its own behavior, which helps reduce search time and prevents the algorithm from being penalized.

## Lorenz

In [ ]:
#Applies AIFeynman to the dynamics
# Note: Due to the supervised approach adopted in this study, each expression considers only the variables
# relevant to describing the dynamics, based on prior knowledge of the system. Thus, since each model uses
# only the variables that influence a specific target variable, it is necessary to fit a separate model
# for each one.

#X axis
run_aifeynman("/home/", "dados_lorenzX.txt", 30,"7ops.txt", polyfit_deg=2, NN_epochs=40)
# When the solution is verified, interrupt the execution before finalizing the process.
# 0.000000000000+(x1+(x1+(-(x0+x0))))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/dados_lorenzX.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         200  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     200
Shuffling mystery data..
 Searching for best fit...
     25.907383630042     -0.088232949943                      0               1             0.0000          5181.4767             0.4845             2.8882             2.8628           200.0000
     24.568458687988      0.509480675953            

In [ ]:
#Y axis
from aifeynman import run_aifeynman
run_aifeynman("/home/", "dados_lorenzY.txt", 60,"7ops.txt", polyfit_deg=2, NN_epochs=40)
# When the solution is verified, interrupt the execution before finalizing the process.
# 0.000000000000+(x0+(-(x1+(x2*x0))))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/dados_lorenzY.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       3
Functions used..........               +*/>~R0
 Arity            0 : 0abc
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         200  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     200
Shuffling mystery data..
 Searching for best fit...
     26.089009741582     -0.094753611015                      0               1             0.0000          5217.8019             0.4716             1.9052             2.8453           200.0000
     26.002897816146      0.582856277129           

     21.397968314177     -0.437269019346                 c>a>>/           26324            14.6841          4294.2778             0.6895             3.6680             5.2883           124.5125
     20.657227769836      0.000892137898                ccca***           39376            15.2650          4146.7106             0.5816             3.3991             4.9623           124.8472
     20.391591947136      0.001166267730                cccb***           39440            15.2674          4093.5858             0.4043             1.9988             4.8656           124.9218
     20.328069699022      0.000486751244               cccbR***          297924            18.1846          4083.7985             0.4891             2.5853             5.3459           101.5610
     19.922782159829      0.000184261058               cccc>/**          298244            18.1861          4002.7426             0.6450             3.4998             6.1485           101.6479
     19.922782159829      0.00

In [ ]:
#Z axis
from aifeynman import run_aifeynman
run_aifeynman("/home/", "dados_lorenzZ.txt", 30, "7ops.txt",  polyfit_deg=2,NN_epochs=1000)
# When the solution is verified, interrupt the execution before finalizing the process.
# 0.003971338265+((x1+(-x0))*(x1+x0))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/dados_lorenzZ.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       3
Functions used..........               +*/>~R0
 Arity            0 : 0abc
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         200  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     200
Shuffling mystery data..
 Searching for best fit...
     26.071741653697     -0.085475961658                      0               1             0.0000          5214.3483             0.2210             1.3145             2.4332           200.0000
     25.621246316270      0.614265604837           

Checking polyfit 

Pareto frontier in the current branch:

Complexity #  MDL Loss #  Expression
42.96 26.07 -0.085475961658+0
46.48 23.59 0.122815671719+(-x2)
50.44 21.93 0.006623001567+(x2*(-(x2+x2)))
51.14 20.61 0.003971338265+((x1+(-x0))*(x1+x0))
54.31 20.61 0.003971338265+((x0*(-x0))+(x1*x1))
59.14 20.54 0.008522112515+(x2/((x2/(x1+(-x0)))+1))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_squared/dados_lorenzZ.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       3
Functions used..........               +*/>~R0
 Arity            0 : 0abc
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         200  rows read from file mystery.dat                                                                                                                                                                                                             

Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       3
Functions used..........               +*/>~R0
 Arity            0 : 0abc
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         200  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     200
Shuffling mystery data..
 Searching for best fit...
     26.034063215322     -0.085268703011                      0               1             0.0000          5206.8126             0.1812             0.8653             2.4008           200.0000
     25.638559073011      0.614380841549                     a~              10             3.3219          5131.0337             0.1596             0.9649             2.4659  

     21.476186091012     -3.170521091298               ac>/cc**         1091874            20.0584          4315.2956             0.2736             0.8797             5.5558           119.1893
Checking polyfit 

Pareto frontier in the current branch:

Complexity #  MDL Loss #  Expression
42.93 25.91 sqrt(-0.992949177644+(0+1))
46.48 23.59 0.122815671719+(-x2)
50.44 21.93 0.006623001567+(x2*(-(x2+x2)))
51.14 20.61 0.003971338265+((x1+(-x0))*(x1+x0))
54.31 20.61 0.003971338265+((x0*(-x0))+(x1*x1))
59.14 20.54 0.008522112515+(x2/((x2/(x1+(-x0)))+1))
84.88 19.87 tan(-0.185500392463+(x1+(-sqrt((x2*(x0+x0))))))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_cos/dados_lorenzZ.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       3
Functions used..........               +*/>~R0
 Arity            0 : 0abc
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery

Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       3
Functions used..........               +*/>~R0
 Arity            0 : 0abc
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         200  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     200
Shuffling mystery data..
 Searching for best fit...
     25.891049589762      0.918075211795                      0               1             0.0000          5178.2099             0.1492             0.6715             2.3584           200.0000
     25.782744806068      1.617757666376                     a~              10             3.3219          5159.8709             0.1511             0.7712             2.4688  

...

     22.632672649031     -0.640977959044              0>aaa**>/         1216335            20.2141          4546.7486             0.0736             0.3465             3.4604            85.8521
     22.329623038890     -0.628600875660             aaaaa***>/         1975842            20.9140          4486.8386             0.1030             0.7087             3.0568            77.5103
Checking polyfit 

Pareto frontier in the current branch:

Complexity #  MDL Loss #  Expression
42.93 25.91 sqrt(-0.992949177644+(0+1))
49.67 25.3 -0.165317443575*((0+1)/x0)
52.12 25.24 0.023961267363*(x0/((-x0)+1))
54.14 24.25 0.257819010812*((0+1)/(x0*(-x0)))
55.63 22.82 0.136017577635*((0+1)/(x0*((-x0)+1)))
55.73 22.81 0.582805993460*((0+1)/(x0*(x0*(-x0))))
61.48 22.72 -0.642918350541*((0+1)/((x0*(x0*x0))+1))
65.68 22.5 -0.635115528324*(x0/((x0*(x0*(x0*x0)))+1))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_cos/dados_lorenzZ.txt_train-

     21.963342223205      0.838383904085               aaa>+/>R           64918            15.9863          4408.6548             0.0858             0.7562             2.5691            44.9548
     20.611714868368      0.799911865171               aa>R+a>/          119010            16.8607          4139.2037             0.0821             0.7336             3.3565            40.8381
     20.453194334973      0.775833220151               aRa>>>/>          205718            17.6503          4108.2892             0.0848             0.7494             3.6668            41.5914
     20.083709575273      0.650319072439              aRa>>>/R>         1418288            20.4357          4037.1776             0.0868             0.7600             4.2674            39.9491
     19.983824300569      0.771165352796             aaaR>*>>/>         2769290            21.4011          4018.1659             0.0795             0.7163             4.1347            39.6619
     19.983580582766      0.83

     22.152326756951     -2.903963102674             aaRa>*>>~/         4423634            22.0768          4452.5422             0.0590             0.2209             3.6237            46.3828
Checking polyfit 

Pareto frontier in the current branch:

Complexity #  MDL Loss #  Expression
42.93 25.91 sqrt(-0.992949177644+(0+1))
49.67 25.3 -0.165317443575*((0+1)/x0)
52.12 25.24 0.023961267363*(x0/((-x0)+1))
54.14 24.25 0.257819010812*((0+1)/(x0*(-x0)))
55.63 22.82 0.136017577635*((0+1)/(x0*((-x0)+1)))
55.73 22.81 0.582805993460*((0+1)/(x0*(x0*(-x0))))
61.48 22.72 -0.642918350541*((0+1)/((x0*(x0*x0))+1))
65.68 22.5 -0.635115528324*(x0/((x0*(x0*(x0*x0)))+1))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_inverse/dados_lorenzZ.txt_train-gen_sym-translated_divide
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity        

Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         200  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     200
Shuffling mystery data..
 Searching for best fit...
     26.048909980212     -0.085371916450                      0               1             0.0000          5209.7820             0.1890             0.9001             2.4109           200.0000
     25.945704446409     -1.812328510785                    a>R              34             5.0875          5194.2284             0.1230             0.3538             2.3323    

Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         200  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     200
Shuffling mystery data..
 Searching for best fit...
     23.788801403557      0.007050822356                      0               1             0.0000          4757.7603             0.2269             1.9529             2.8751           200.0000
     23.788801403557     -0.992949177644                     0>               3             1.5850          4759.3452             0.2269             1.9529             2.8751    

In [ ]:
#Load the solution
df = pd.read_csv('Lorenz_solution.txt', delim_whitespace=True)

#Separate targets (real derivatives)
targets = [
    df['dxdt'].values,
    df['dydt'].values,
    df['dzdt'].values
]

#Original derivative values
x = df['x'].values
y = df['y'].values
z = df['z'].values

#Predicted derivatives
dxdt_pred = -2.0 * x  + 2.0*y
dydt_pred = -1.0 *x*z + 1.0*x - 1.0*y
dzdt_pred = 0.003971338265+((y+(-x))*(y+x))

y_preds = [dxdt_pred,dydt_pred,dzdt_pred]
feature_names = ['x', 'y', 'z']

In [ ]:
res = error_eval('Lorenz', feature_names, y_preds=y_preds, targets=targets)
res

,dynamics,axis,mae,mse,r2,rmse,wilcoxon_raw_value,p-value,is_difference
0,Lorenz,x,1.139921e-16,3.622771e-32,1.000000,1.903358e-16,0.0,1.000000,0
1,Lorenz,y,6.683543e-17,1.107217e-32,1.000000,1.052244e-16,0.0,1.000000,0
2,Lorenz,z,2.437311e-01,6.750527e-01,-15.662574,8.216159e-01,11249.0,0.000105,1


## Lotka-Volterra

In [ ]:
#Applies AIFeynman to the dynamics
# Note: Due to the supervised approach adopted in this study, each expression considers only the variables
# relevant to describing the dynamics, based on prior knowledge of the system. Thus, since each model uses
# only the variables that influence a specific target variable, it is necessary to fit a separate model
# for each one.

#u axis
from aifeynman import run_aifeynman
run_aifeynman("/home/", "dados_LotkU.txt", 60,"7ops.txt", polyfit_deg=2, NN_epochs=100)
# Was the dynamic checked in an initial iteration? Stop the process.
# 13.32 0.0 0.500000000000*(x0*(((((-x1)+1)+1)+1)+1))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_LotkU.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
          60  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......      60
Shuffling mystery data..
 Searching for best fit...
     30.943649293543     -0.110443741057                      0               1             0.0000          1856.6190            13.4689            33.6238             4.9111            60.0000
     30.795503801052     -0.7161221

     30.684397510040     -0.480331150002                aaRa>**           38591            15.2360          1856.2998           449.0764          1777.8168             7.2796            48.0427
     30.684397510040     -0.480331150002                aaR*a>*           55925            15.7712          1856.8351           449.0764          1777.8168             7.2796            47.6839
     30.684397510040     -1.480331150002               aaRa>**>          314153            18.2611          1859.3250           449.0764          1777.8168             7.2796            46.1378
     30.684397510040     -0.480331150002              aaaRa>//*         1250774            20.2544          1861.3182           449.0764          1777.8168             7.2796            44.5282
     30.659292954059     -0.640180327249              aabR>R>**         1345958            20.3602          1859.9178           287.0617          1018.9228             7.3287            44.5458
     30.628441552934     -1.92

In [ ]:
#v axis
from aifeynman import run_aifeynman
run_aifeynman("/home/", "dados_LotkV.txt", 60,"7ops.txt", polyfit_deg=2, NN_epochs=300)
# Was the dynamic checked in an initial iteration? Stop the process.
# 59.39 25.54 -0.201604790782*(x1*((((((-x0)+1)+1)+1)+1)+1))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_LotkV.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
          60  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......      60
Shuffling mystery data..
 Searching for best fit...
     30.863013626778     -1.248426349776                      0               1             0.0000          1851.7808             6.7513            17.1474             4.3809            60.0000
     30.273604555478     -4.5710098

     29.576715953201     -0.719788640572               bbbRRR/*          166602            17.3460          1791.9490            69.3828           220.0228             6.8038            46.5710
     29.576715953201     -0.719788640572               bbRRRb//          378741            18.5309          1793.1338            69.3828           220.0228             6.8038            46.0196
     29.530433805556     -0.783332522626               bb*>a>R/          581502            19.1494          1790.9755            75.8607           243.7227             6.2282            45.2546


In [ ]:
#Load the solution
df = pd.read_csv('LotkV_solution.txt', delim_whitespace=True)

#Separate targets (real derivatives)
targets = [
    df['dudt'].values,
    df['dvdt'].values
]

#Original derivative values
u = df['u'].values
v = df['v'].values

#Predicted derivatives
dudt_pred = -0.5*u*v + 2.0*u
dvdt_pred = 0.2*u*v - 1.0*v

y_preds = [dudt_pred,dvdt_pred]
feature_names = ['u', 'v']

In [ ]:
res = error_eval('Lotka-Volterra', feature_names, y_preds=y_preds, targets=targets)
res

,dynamics,axis,mae,mse,r2,rmse,wilcoxon_raw_value,p-value,is_difference
0,Lotka-Volterra,u,1.159073e-15,7.342194e-30,1.000000,2.709648e-15,0.0,1.000000e+00,0
1,Lotka-Volterra,v,3.223242e-01,3.288714e-01,0.991431,5.734731e-01,0.0,5.280373e-14,1


## Simple pendulum

In [ ]:
#Applies AIFeynman to the dynamics
# Note: Due to the supervised approach adopted in this study, each expression considers only the variables
# relevant to describing the dynamics, based on prior knowledge of the system. Thus, since each model uses
# only the variables that influence a specific target variable, it is necessary to fit a separate model
# for each one.

# theta
run_aifeynman("/home/", "dados_pendT.txt", 60,"14ops.txt", polyfit_deg=1, NN_epochs=600)
# 𝑥0 # Was the dynamic checked in an initial iteration? Stop the process.


Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_pendT.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........        +*-/><~\RPSCLE
 Arity            0 : Pa
 Arity            1 : ><~\RSCLE
 Arity            2 : +*-/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
     30.204872228305     -3.268607741114                      P               1             0.0000         24163.8978             1.6845             2.5220             1.6731           800.0000
      0.000000000000      0.0

     25.653772345324     -0.008373480911                 aPL\*S           46456            15.5036         20538.5215             0.1536             0.3165             2.4773            78.8787
     25.398391028081     -0.001727304422                 aPLS*S           46528            15.5058         20334.2186             0.1756             0.3574             2.6580            78.9118
     25.189469992183     -0.999973747924                 aaC<+E           49946            15.6081         20167.1841             0.1729             0.3149             4.2115            77.9181
     25.189469992183     -0.999973747924                 aCa<+E          102366            16.6434         20168.2194             0.1729             0.3149             4.2115            96.2922
     25.174642804444     -0.000007387560                PaPa/+/          270327            18.0443         20157.7586             0.1963             0.3280             4.3396            93.6497
     24.607147508826      0.00

In [ ]:
# omega
from aifeynman import run_aifeynman
run_aifeynman("/home/", "dados_pendO.txt", 60, "14ops.txt",NN_epochs=600)
# Was the dynamic checked in an initial iteration? Stop the process.
# 57.8 22.17 -9.805524248761*sin(x0)

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_pendO.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........        +*-/><~\RPSCLE
 Arity            0 : Pa
 Arity            1 : ><~\RSCLE
 Arity            2 : +*-/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
     31.804064223458     -2.897656428555                      P               1             0.0000         25443.2514             5.0132             7.1736             1.7102           800.0000
     31.640064334733      0.2

     33.202070542996     13.009756965121                  PaEE*            2863            11.4833         26573.1397            16.1380            29.9986             1.9130           619.8788
     32.920511412532     19.538715796663                  aP>*E            4702            12.1991         26348.6082            15.0164            28.3885             2.0176           618.0634
     31.833186599968    -20.468355315170                  PaC/E            4799            12.2285         25478.7778             6.9583            16.5314             1.8826           618.1550
     31.691947514627    -48.397395639586                 PaC\+E           50017            15.6101         25369.1681             7.1329            14.5055             1.9003           224.5480
     30.923143001476    -13.642256327044                 PaSC/E           51093            15.6408         24754.1552             4.4687             9.4984             2.1577           223.7178
     30.664979669356    -61.91

     25.870471465694      1.570776173624                PPaS**\          295121            18.1709         20714.5481             6.9590           153.1002             6.7793           110.1719
     25.740775053974     -1.403002797425                PaP*EE/          312999            18.2558         20610.8758             0.2526             0.6927             2.5174           121.8573
     25.060025412825     -1.285484322210                aa>S</E          583796            19.1551         20067.1754             0.1619             0.4742             3.4721            90.0516
     24.816268370572      0.001486653405               aaCRL~L*         5683486            22.4383         19875.4530             0.0818             0.1969             2.1245            73.2918
     24.816268370572      0.001486653405               aaC\RLL*         5697058            22.4418         19875.4565             0.0818             0.1969             2.1245            73.1550
Checking for brute force * 

T

     23.446157405202      0.003174087508                PPaS**C          298577            18.1877         18775.1137             0.0201             0.0372             1.8323           255.4300
     23.446157405202     -0.996825912492               PPaS**C>         3761571            21.8429         18778.7688             0.0201             0.0372             1.8323            38.6036
     23.446157405202      1.003174087508               PPaS**C<         3771939            21.8469         18778.7728             0.0201             0.0372             1.8323            38.5612
Checking for brute force * 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_cos/dados_pendO.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........        +*-/><~\RPSCLE
 Arity            0 : Pa
 Arity            1 : ><~\RSCLE
 Arity            2 : +*-/
Loading mystery data....
         800  rows r

     29.007290749041     -0.006705792048                  a\>>S           17418            14.0883         11907.0775           362.6120          1018.1620             9.1067           283.9467
     28.962449722371     -0.005902710715                  a>L\C           19482            14.2499         11888.8542           362.6122          1018.1622             9.2078           285.1895
     28.961621152778      0.005899439831                 a>L\C~          187008            17.5127         11891.7774           362.6122          1018.1622             9.2100           272.9229
     27.705559301566      0.995551105440                aPP~**E          300854            18.1987         11377.4780           400.3463          1283.9094             8.0957           263.2256
Checking polyfit 

Pareto frontier in the current branch:

Complexity #  MDL Loss #  Expression
15.97 0.0 -9.800000000000*sin(x0)

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve re

Checking polyfit 

Pareto frontier in the current branch:

Complexity #  MDL Loss #  Expression
0.0 28.17 1/(666.000000000000+(sin(pi))**(-1))
15.97 0.0 -9.800000000000*sin(x0)

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_sin/dados_pendO.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........        +*-/><~\RPSCLE
 Arity            0 : Pa
 Arity            1 : ><~\RSCLE
 Arity            2 : +*-/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
     28.730227937934     -3.139506301091        

In [ ]:
#Load the solution
df = pd.read_csv('Pendulum_solution.txt', delim_whitespace=True)

#Separate targets (real derivatives)
targets = [
    df['dthetadt'].values,
    df['domegadt'].values
]

#Original derivative values
theta = df['T'].values
omega = df['O'].values


#Predicted derivatives
dthetadt_pred = 1.0 * omega
domegadt_pred = -9.8 * np.sin(theta)

y_preds = [dthetadt_pred, domegadt_pred]
feature_names = ['theta', 'omega']


In [ ]:
res = error_eval('Pendulum', feature_names, y_preds=y_preds, targets=targets)
res

,dynamics,axis,mae,mse,r2,rmse,wilcoxon_raw_value,p-value,is_difference
0,Pendulum,theta,0.000000e+00,0.000000e+00,1.0,0.000000e+00,0.0,1.0,0
1,Pendulum,omega,3.111417e-16,3.215513e-31,1.0,5.670550e-16,0.0,1.0,0


## SIR

In [ ]:
#Applies AIFeynman to the dynamics
# Note: Due to the supervised approach adopted in this study, each expression considers only the variables
# relevant to describing the dynamics, based on prior knowledge of the system. Thus, since each model uses
# only the variables that influence a specific target variable, it is necessary to fit a separate model
# for each one.

#S compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/",'dados_sirS.txt', 60,"7ops.txt", polyfit_deg=2, NN_epochs=400)
# Was the dynamic checked in an initial iteration? Stop the process.
# -0.300000000000*(x1*x0)

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_sirS.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
     21.825611325021     -0.002571142178                      0               1             0.0000         17460.4891             0.0146             0.0390             2.1793           800.0000
     21.825611325021     -2.00257114

     14.719652436619      1.000006474567                 b>~>b/           17088            14.0607         11789.7826             0.0005             0.0017             3.3067           233.9658
     14.080133581618     -0.000000044176                bbbb***           19020            14.2152         11278.3221             0.0022             0.0075             6.0013           232.3569
     12.010152655117      0.000000029550              bbbaa****          972525            19.8914          9628.0135             0.0009             0.0033             5.9449           216.7131


In [ ]:
#I compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/",'dados_sirI.txt',  60,"7ops.txt", polyfit_deg=2, NN_epochs=400)
# Was the dynamic checked in an initial iteration? Stop the process.
# #-0.100000000000*(x1*((-(x0+(x0+x0)))+1))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_sirI.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
     21.486746033276     -0.000866320655                      0               1             0.0000         17189.3968             0.0082             0.0209             2.2753           800.0000
     21.486746033275     -2.00086632

     14.745740724146      1.000016353688                 a>~>a/           17006            14.0538         11810.6463             0.0001             0.0004             2.0732           230.6430
     14.745740723934      1.000016353688                 b>~>b/           17088            14.0607         11810.6533             0.0001             0.0004             2.0732           230.0710
     14.229210613189      0.000000141008                bbba***           18993            14.2132         11397.5817             0.0032             0.0103             5.8187           227.7693
     14.229210613169     -0.999999858992               bbba***>          150213            17.1966         11400.5651             0.0032             0.0103             5.8187           231.9499


In [ ]:
#R compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/",'dados_sirR.txt', 30,"7ops.txt", polyfit_deg=2, NN_epochs=4000)
# Was the dynamic checked in an initial iteration? Stop the process.
#  # sqrt(0.010000000000*(x0*x0))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_sirR.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
     22.135902076251      0.004461095949                      0               1             0.0000         17708.7217             0.0109             0.0257             1.8815           800.0000
     20.815993961613      0.000575491

     13.656097992509     -1.414213504816              aaa**>R>R          576024            19.1358         10944.0142             0.0012             0.0039             5.2772           290.8082
     13.499880953274     -1.189206953854              aaa**>>RR          576888            19.1379         10819.0427             0.0010             0.0032             5.0870           290.7070
     13.351765156517     -0.999999765598              aaa**>RRR          577320            19.1390         10700.5511             0.0008             0.0025             4.9168           290.7060
Checking for brute force * 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_squared/dados_sirR.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read 

     19.824190516255      0.650954029753                a~>R>R>           43584            15.4115         15874.7639             0.0030             0.0077             2.0453            63.1649
     18.718939815440      0.717672968814                a~>>RR>           43656            15.4139         14990.5657             0.0014             0.0037             2.0497            63.1518
     16.638263970161      0.785401423553                a~>RRR>           43692            15.4151         13326.0263             0.0015             0.0043             3.2642            63.1448
     16.260366278462      1.110720826961                a~>RR>R           44340            15.4363         13023.7293             0.0014             0.0039             3.4784            62.6862
     15.832211188319      1.320874403100                a~>R>RR           44556            15.4433         12681.2123             0.0011             0.0031             3.6319            62.4805
     15.205777829045      1.44

      5.065612595110      0.100000331693             aaa>~+>R>/         2760802            21.3967          4073.8867             0.0000             0.0000             4.2249            50.6523
Checking polyfit 

Pareto frontier in the current branch:

Complexity #  MDL Loss #  Expression
15.91 21.97 sqrt(0.010000000000*(x0*x0))
42.08 20.82 0.000575491378+(x0*x0)
47.94 5.85 sin(0.100000331693*x0)

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_atan/dados_sirR.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                   

     14.594988068805     -4.000009953991                 0>>>>>            6555            12.6784         11688.6688             0.0002             0.0004             2.5214           324.1062
     14.594659444590      0.999990046009                aa>~+>R           18072            14.1415         11689.8690             0.0002             0.0004             2.5220           321.2836
     13.424614126607      0.999999899276               aaaa~***           45334            15.4683         10755.1596             0.0023             0.0078             5.8174           302.0898
     13.424614126607     -0.000000100724              aaaa~***>          306640            18.2262         10757.9175             0.0023             0.0078             5.8174           290.4209
Checking for brute force * 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_cos/dados_sirR.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variab

     15.508092097975     -0.598046938640             a0>>R+>R>R         5768860            22.4599         12428.9335             0.0005             0.0014             2.9897            18.3128
     15.508092097975     -0.598046938640             a>0>>R+R>R         9838768            23.2300         12429.7037             0.0005             0.0014             2.9897            17.3489
     15.193660188823     -0.090530328335             a>a>R*>RRR         9999118            23.2534         12178.1815             0.0002             0.0006             2.3527            17.3342
     14.933423817255     -0.479624616506             a>a>*>RR>R        10311634            23.2978         11970.0368             0.0001             0.0001             1.7087            17.0813
Checking for brute force * 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_exp/dados_sirR.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variab

Checking polyfit 

Pareto frontier in the current branch:

Complexity #  MDL Loss #  Expression
6.32 38.01 1/(10.000000000000*((0+1)/x0))
15.91 21.97 sqrt(0.010000000000*(x0*x0))
42.08 20.82 0.000575491378+(x0*x0)
45.33 20.82 tan(0.000575491301+(x0*x0))
47.94 6.74 tan(0.099999312279*(x0+sqrt(((x0+(-(x0+1)))+1))))
47.94 6.74 tan(-0.099999315021*(-(x0+sqrt(((x0+(-(x0+1)))+1)))))
47.94 6.73 tan(0.099999336407*x0)
47.94 6.73 tan(-0.099999336629*(-x0))
47.94 5.85 sin(0.100000331693*x0)

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_log/dados_sirR.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                              

In [ ]:
#Load the solution
df = pd.read_csv('SIR_solution.txt', delim_whitespace=True)

#Separate targets (real derivatives)
targets = [
    df['dSdt'].values,
    df['dIdt'].values,
    df['dRdt'].values
]

#Original derivative values
S = df['S'].values
I = df['I'].values
R = df['R'].values

#Predicted derivatives
dSdt_pred = -0.3 * S * I
dIdt_pred = 0.3 * S * I - 0.1 * I
dRdt_pred = 0.1 * I

y_preds = [dSdt_pred, dIdt_pred, dRdt_pred]
feature_names = ['S', 'I', 'R']


In [ ]:
res = error_eval('SIR',feature_names, y_preds=y_preds, targets=targets)
res


,dynamics,axis,mae,mse,r2,rmse,wilcoxon_raw_value,p-value,is_difference
0,SIR,S,1.000000e-18,6.692663e-36,1.0,2.587018e-18,0.0,1.0,0
1,SIR,I,9.191866e-19,4.235394e-36,1.0,2.058007e-18,0.0,1.0,0
2,SIR,R,5.651810e-19,1.645831e-36,1.0,1.282899e-18,0.0,1.0,0


## SIS

In [ ]:
#S compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/","dados_sisS.txt", 60, "7ops.txt", polyfit_deg=2, NN_epochs=400)
# Was the dynamic checked in an initial iteration? Stop the process.
# 18.52 0.0 -0.100000000000*(x1*(x0+(x0+(-x1))))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_sisS.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
       80000  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......   80000
Shuffling mystery data..
 Searching for best fit...
     20.950871836633     -0.000872822468                      0               1             0.0000       1676069.7469             0.0117             0.0325             2.3669         80000.0000
     20.950871836633     -1.00087282

Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
       80000  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......   80000
Shuffling mystery data..
 Searching for best fit...
     12.603117244214      0.000000761428                      0               1             0.0000       1008249.3795             0.0003             0.0011             4.1334         80000.0000
     12.603117244129     -0.999999238572                    ba+              18             4.1699       1008253.5495             0.0003             0.0011             4.1334   

In [ ]:
#I compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/","dados_sisI.txt", 60, "7ops.txt", polyfit_deg=2, NN_epochs=400)
# Was the dynamic checked in an initial iteration? Stop the process.

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_sisI.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
       80000  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......   80000
Shuffling mystery data..
 Searching for best fit...
     20.951136673323      0.000871737596                      0               1             0.0000       1676090.9339             0.0117             0.0325             2.3728         80000.0000
     20.951136673323     -0.99912826

Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
       80000  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......   80000
Shuffling mystery data..
 Searching for best fit...
     12.604601614292      0.000000759926                      0               1             0.0000       1008368.1291             0.0003             0.0011             4.1408         80000.0000
     12.604601614243     -0.999999240074                     0>               4             2.0000       1008370.1291             0.0003             0.0011             4.1408   

In [ ]:
#Load the solution
df = pd.read_csv('SIS_solution.txt', delim_whitespace=True)

#Separate targets (real derivatives)
targets = [
    df['dSdt'].values,
    df['dIdt'].values
]

#Original derivative values
S = df['S'].values
I = df['I'].values

#Predicted derivatives
dSdt_pred = -0.2*S*I+0.1*I**2
dIdt_pred = 0.2*S*I - 0.1*I**2

y_preds = [dSdt_pred, dIdt_pred]
feature_names = ['S', 'I']

In [ ]:
res = error_eval('SIS', feature_names, y_preds=y_preds, targets=targets)
res

,dynamics,axis,mae,mse,r2,rmse,wilcoxon_raw_value,p-value,is_difference
0,SIS,S,6.561411e-18,9.054127e-35,1.0,9.515318e-18,0.0,1.0,0
1,SIS,I,6.561411e-18,9.054127e-35,1.0,9.515318e-18,0.0,1.0,0


## SEIR

In [ ]:
#Applies AIFeynman to the dynamics
# Note: Due to the supervised approach adopted in this study, each expression considers only the variables
# relevant to describing the dynamics, based on prior knowledge of the system. Thus, since each model uses
# only the variables that influence a specific target variable, it is necessary to fit a separate model
# for each one.

#S compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/","dados_seirS.txt", 240, "7ops.txt", polyfit_deg=2, NN_epochs=600)
# Was the dynamic checked in an initial iteration? Stop the process.
# 13.32 0.0 -0.300000000000*(x1*x0)

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_seirS.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
      8.691057932729     -0.000000251680                      0               1             0.0000          6952.8463             0.0000             0.0003             2.6672           800.0000
      8.691057932392     -1.0000002

      0.071612925592      0.000000000000                bbbb+**           18939            14.2091            71.4994             0.0000             0.0000             0.5370           222.1155
      0.071377239259      0.000000000000                bba~>**           21090            14.3643            71.4661             0.0000             0.0000             0.5366           225.2579
      0.070707331643     -1.414213562373                bb*>R>R           82947            16.3399            72.9058             0.0000             0.0000             0.5331           220.7069
      0.057395543200     -1.189207115003                bb*>>RR           83271            16.3455            62.2620             0.0000             0.0000             0.4625           220.5820
      0.042976875606     -1.000000000000                bb*>RRR           83433            16.3483            50.7298             0.0000             0.0000             0.3760           220.6351
      0.034302758138     -1.55

      1.839186581042      1.570796325111              ba>>RR>>/         3082527            21.5557          1492.9049             0.0000             0.0000             2.1815            10.7019
      1.762018465886      0.381589213337              bb+a/>>RR         4405206            22.0708          1431.6856             0.0000             0.0000             2.1828            10.3160
      0.717259174212      1.570796326411             baa>>/>>>/        11788821            23.4909           597.2983             0.0000             0.0000             1.5556             9.7987
      0.705801093229      1.570796326423            baaa>+/>>>/        66666024            25.9904           590.6313             0.0000             0.0000             1.5398             5.3589
      0.682029764080      1.570796326447            b0>a>>/>>>/       161793951            27.2696           572.8934             0.0000             0.0000             1.5067             4.8196
      0.680410951913      1.57

In [ ]:
#E compartment:
from aifeynman import run_aifeynman
run_aifeynman("/home/","dados_seirE.txt",300 , "7ops.txt", polyfit_deg=2, NN_epochs=4000)
# Was the dynamic checked in an initial iteration? Stop the process.
# 20.25 0.0 -0.100000000000*(x1+(x1+(x2*(-(x0+(x0+x0))))))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_seirE.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       3
Functions used..........               +*/>~R0
 Arity            0 : 0abc
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
      9.021540921239     -0.000000188485                      0               1             0.0000          7217.2327             0.0000             0.0003             2.8098           800.0000
      9.021540921108     -1.000000

      0.876979234137     -0.099928036711             bbca>>~*++        21265855            24.3420           426.8755             0.0000             0.0000             1.5920            11.7302
      0.876979234137     -0.099928036711             ca>>~*bb++        77863788            26.2144           428.7479             0.0000             0.0000             1.5920             6.4971
      0.597106987304     -0.099964005404            bbcaa>+~*++       170407307            27.3444           301.4165             0.0000             0.0000             1.2170             5.1981
      0.365894427343     -0.199964024640            bcaaa>/+~*+       171233647            27.3514           195.2969             0.0000             0.0000             0.8655             5.1902
      0.365894427343     -0.199964024640            bacca>/+~*+       171233799            27.3514           195.2969             0.0000             0.0000             0.8655             5.1902
      0.040872461880     -0.19

Checking polyfit 

Pareto frontier in the current branch:

Complexity #  MDL Loss #  Expression
0.0 0.1 sqrt(0.000000000000+0)
20.25 0.0 -0.100000000000*(x1+(x1+(x2*(-(x0+(x0+x0))))))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_acos/dados_seirE.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       3
Functions used..........               +*/>~R0
 Arity            0 : 0abc
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
      9.020165601943      1.570796513709     

      3.349432354309      0.460268077687             ac>>Rc>/>+        75249474            26.1652          2705.7111             0.0000             0.0000             2.2861            16.9035
      3.349432352735      0.460268077687             ac>>Rc>/+>        75389442            26.1679          2705.7137             0.0000             0.0000             2.2861            16.8829
Checking polyfit 

Pareto frontier in the current branch:

Complexity #  MDL Loss #  Expression
0.0 0.1 sqrt(0.000000000000+0)
20.25 0.0 -0.100000000000*(x1+(x1+(x2*(-(x0+(x0+x0))))))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_asin/dados_seirE.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       3
Functions used..........               +*/>~R0
 Arity            0 : 0abc
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file

In [ ]:
#I compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/","dados_seirI.txt", 3600, "7ops.txt", polyfit_deg=2, NN_epochs=600)
# Was the dynamic checked in an initial iteration? Stop the process.
# 12.32 0.0 0.000000000000+((x0/(((((0+1)+1)+1)+1)+1))+(-x1))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_seirI.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
      9.766093688915     -0.000000566037                      0               1             0.0000          7812.8750             0.0001             0.0010             2.2270           800.0000
      8.478997960593      0.0000000

      4.945129787483     -0.492131800273             b>b>*a>R~+        45122919            25.4274          1538.6371             0.0000             0.0000             4.5979            40.5427
      4.775372178298     -0.336104211102            bbba>R~>+++        61233285            25.8678          1487.1317             0.0000             0.0000             4.2121            39.5342
      4.775338963211     -0.504156318297            bba0>>>~/++        61247343            25.8681          1487.1219             0.0000             0.0000             4.2121            39.5429
      4.775338963211     -0.336104212198            babb+~/>>>*        73515987            26.1316          1487.3853             0.0000             0.0000             4.2121            39.4553
      3.491495019722     -1.002871755901            ba0>>RR~>*+        91370826            26.4452          1094.8427             0.0000             0.0000             3.8673            39.0390
      3.491132709581     -1.00

In [ ]:
#R compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/","dados_seirR.txt",60 , "7ops.txt", polyfit_deg=2, NN_epochs=600)
# Was the dynamic checked in an initial iteration? Stop the process.

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_seirR.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
     10.470640654346      0.000000863333                      0               1             0.0000          8376.5125             0.0001             0.0010             2.7145           800.0000
      0.000000000000      0.00000000

      6.077770757250      0.523598789180              a0>>>a>/+          742198            19.5014          4881.7180             0.0000             0.0000             2.6750             9.2313
      4.728962856412     -0.785398155519              aa~>RR>~+          765388            19.5458          3802.7161             0.0000             0.0000             2.6348             9.1645
      4.018664657732     -1.010955494508              a0>>R>R~+          765814            19.5466          3234.4784             0.0000             0.0000             2.5923             9.1686
      4.018265270943      1.010955494498              a~0>>R>R+         1394520            20.4113          3235.0236             0.0000             0.0000             2.5925             6.9596
      2.743774935768      0.574951358463              0>>>Ra>/>         1886403            20.8472          2215.8672             0.0000             0.0000             2.4529             6.3239
      2.738423308816      0.57

In [ ]:
#Load the solution
df = pd.read_csv('SEIR_solution.txt', delim_whitespace=True)

#Separate targets (real derivatives)
targets = [
    df['dSdt'].values,
    df['dEdt'].values,
    df['dIdt'].values,
    df['dRdt'].values
]

#Original derivative values
S = df['S'].values
E = df['E'].values
I = df['I'].values
R = df['R'].values

#Predicted derivatives
dSdt_pred = -0.3*S*I
dEdt_pred = 0.3*S*I -0.2*E
dIdt_pred = 0.2 * E - 1.0 * I
dRdt_pred = 1.0*I

y_preds = [dSdt_pred,dEdt_pred,dIdt_pred,dRdt_pred]
feature_names = ['S', 'E', 'I', 'R']

In [ ]:
res = error_eval('SEIR', feature_names, y_preds=y_preds, targets=targets)
res

,dynamics,axis,mae,mse,r2,rmse,wilcoxon_raw_value,p-value,is_difference
0,SEIR,S,5.507569e-22,1.748074e-41,1.0,4.180998e-21,0.0,1.0,0
1,SEIR,E,6.633029e-22,1.116843e-41,1.0,3.341920e-21,0.0,1.0,0
2,SEIR,I,9.364454e-22,4.314589e-41,1.0,6.568553e-21,0.0,1.0,0
3,SEIR,R,0.000000e+00,0.000000e+00,1.0,0.000000e+00,0.0,1.0,0


## SEIRD

In [ ]:
#Applies AIFeynman to the dynamics
# Note: Due to the supervised approach adopted in this study, each expression considers only the variables
# relevant to describing the dynamics, based on prior knowledge of the system. Thus, since each model uses
# only the variables that influence a specific target variable, it is necessary to fit a separate model
# for each one.

#S compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/","dados_seirdS.txt", 3600, "7ops.txt", polyfit_deg=2, NN_epochs=1000)
# Was the dynamic checked in an initial iteration? Stop the process.
# 13.32 0.0 -0.300000000000*(x1*x0)


Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_seirdS.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
      8.457516428726     -0.000000215274                      0               1             0.0000          6766.0131             0.0000             0.0003             2.6082           800.0000
      8.457516428237     -2.000000

Checking polyfit 

Pareto frontier in the current branch:

Complexity #  MDL Loss #  Expression
13.32 0.0 -0.300000000000*(x1*x0)

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_squared/dados_seirdS.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
      0.063238619870      0.000000000000                      0               1             0.00

      1.804430148597      0.381589213426               b>b>*>RR          765213            19.5455          1463.0896             0.0000             0.0000             2.1423            12.5008
      1.708139777371      0.381589213280              bba/>>+RR         2062803            20.9762          1387.4880             0.0000             0.0000             2.1190            11.2715
      1.708139775724      0.381589213280              bba/>+>RR         2141535            21.0302          1387.5420             0.0000             0.0000             2.1190            11.2196
      1.708139775257      0.381589213280              bba/+>>RR         2318682            21.1449          1387.6567             0.0000             0.0000             2.1190            11.0905
      1.654715642834      1.570796325367              ba>>RR>>/         3082527            21.5557          1345.3282             0.0000             0.0000             2.0855            10.8418
      1.604591951268      0.38

      0.054844993678     -0.423725675976            b>0>>R/>>>~       351636945            28.3895            72.2655             0.0000             0.0000             0.4795             1.7490
      0.054844993287     -0.423725675976          aaa+/Rb>*>>>~      9651447836            33.1681            77.0441             0.0000             0.0000             0.4795             1.4517
      0.054844993103     -0.299619298846          b0>>R0>>>*>+~     12969245166            33.5944            77.4704             0.0000             0.0000             0.4795             1.4496
      0.005597715724      0.423725675976          b>bb~*>>R/>>>     20331330378            34.2430            38.7212             0.0000             0.0000             0.0989             1.4466
Checking polyfit 

Pareto frontier in the current branch:

Complexity #  MDL Loss #  Expression
0.0 0.09 sqrt(0.000000000000+0)
13.32 0.0 -0.300000000000*(x1*x0)

Checking for brute force + 

Trying to solve mysteries with b

      0.047542267075     -1.316074012953          ba>>R>~*>>>RR     13392609732            33.6407            71.6745             0.0000             0.0000             0.3040             1.9869
Checking for brute force * 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_asin/dados_seirdS.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Removing problematically
   2.6855679998835153E-004   2.6855681274411505E-007
         4

      0.173760677740      1.116469750024           b>b>>R>R*RR~      2799384954            31.3825           170.3910             0.0000             0.0000             0.8342             3.3606
      0.112631962028     -1.356796379383          abab>>RR*+/>R      4065593366            31.9208           122.0264             0.0000             0.0000             0.7379             2.8223
      0.075416262596      0.000000000020          baaa>>/>>>~/*      4137090741            31.9460            92.2790             0.0000             0.0000             0.4446             2.8018
      0.075416262596      0.000000000020          abaa>>/>>>~/*      4137090743            31.9460            92.2790             0.0000             0.0000             0.4446             2.8018
      0.075416262596      0.000000000020          baa>>/>>>a~//      6647105268            32.6301            92.9631             0.0000             0.0000             0.4446             2.3593
      0.047536761718     -1.31

Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
      8.457513816552      0.999999784726                      0               1             0.0000          6766.0111             0.0000             0.0003             2.6082           800.0000
      8.457455594377      0.999999784726                   bb~*             111             6.7944          6772.7589             0.0000             0.0003             2.6080   

In [ ]:
#E compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/","dados_seirdE.txt", 3600, "7ops.txt", polyfit_deg=2, NN_epochs=600)
# Was the dynamic checked in an initial iteration? Stop the process.
# 20.25 0.0 -0.100000000000*(x1+(x1+(x2*(-(x0+(x0+x0))))))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_seirdE.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       3
Functions used..........               +*/>~R0
 Arity            0 : 0abc
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
      8.930477848010     -0.000000153030                      0               1             0.0000          7144.3823             0.0000             0.0003             2.7726           800.0000
      8.930477847624     -1.00000

      0.019501011629     -0.199999996658            bacc>R+~>*+       192338183            27.5191            36.4115             0.0000             0.0000             0.2217             3.9497
      0.000000000000     -0.100000000000           bbcaaa++~*++      1465478559            30.4487            30.4487             0.0000             0.0000             0.0000             1.8521
 All done: results in results.dat                                                                                                                                                                                                                                                     
Checking polyfit 

Pareto frontier in the current branch:

Complexity #  MDL Loss #  Expression
20.25 0.0 -0.100000000000*(x1+(x1+(x2*(-(x0+(x0+x0))))))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_squared/dados_seirdE.txt_train
Rejection threshold.....    10.000


In [ ]:
#I compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/","dados_seirdI.txt",60, "7ops.txt", polyfit_deg=2, NN_epochs=4000)
# Was the dynamic checked in an initial iteration? Stop the process.
# -0.539112974991*(x1+(x1+(x0/(-((x0+1)+1)))))
#3600, "7ops.txt", polyfit_deg=2, NN_epochs=600)


Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_seirdI.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
        1600  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......    1600
Shuffling mystery data..
 Searching for best fit...
      9.652505436328     -0.000000475957                      0               1             0.0000         15444.0087             0.0001             0.0010             2.2519          1600.0000
      9.652505436218     -1.000000

Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
        1600  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......    1600
Shuffling mystery data..
 Searching for best fit...
      0.164206254718      0.000000000001                      0               1             0.0000           262.7300             0.0000             0.0000             1.0356          1600.0000
      0.116560068711      0.000000000000                    bb*              30             4.9069           191.4030             0.0000             0.0000             0.6335   

Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
        1600  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......    1600
Shuffling mystery data..
 Searching for best fit...
      9.652720326515      1.570796802627                      0               1             0.0000         15444.3525             0.0001             0.0010             2.2511          1600.0000
      8.541278432227      1.570796220667                      b               3             1.5850         13667.6305             0.0000             0.0001             3.1422   

      8.539662845128      0.000000105858                 bb~>~*            5835            12.5105         13675.9711             0.0000             0.0001             3.1423           421.5013
      8.539477846442      0.000000105858                baa>+~/           23976            14.5493         13677.7139             0.0000             0.0001             3.1420           393.6415
      8.538026247356      0.000000105799                bba>+~/           23979            14.5495         13675.3915             0.0000             0.0001             3.1440           393.7071
      8.538026247356      0.000000105799                bab>+~/           23985            14.5498         13675.3918             0.0000             0.0001             3.1440           393.8382
      8.275355517925     -0.000000009840                bab/>~/           30600            14.9012         13255.4701             0.0000             0.0006             2.3518           394.6604
      8.154315999337     -1.41

      8.539478169454      0.000000105858                baa>+~/           23976            14.5493         13677.7144             0.0000             0.0001             3.1420           393.6415
      8.538024489033      0.000000105799                bba>+~/           23979            14.5495         13675.3887             0.0000             0.0001             3.1440           393.7071
      8.275355499828     -0.000000009840                bab/>~/           30600            14.9012         13255.4700             0.0000             0.0006             2.3518           394.6604
      8.154315972132     -1.414213651149                b~a>>R+           92745            16.5010         13063.4065             0.0000             0.0001             3.4036           262.1607
      7.524526322933     -1.000000083464                b~a>RR+           92907            16.5035         12055.7456             0.0000             0.0001             3.0919           262.0343
      6.211005689540     -1.41

...

      9.652720321346      3.570796802627               aa+a>~>/          150066            17.1952         15461.5478             0.0001             0.0010             2.2511           492.9335
      9.649790911434      1.570796802627              a0>a~/*>R          638158            19.2836         15458.9490             0.0001             0.0010             2.2603           462.0911
Checking for brute force * 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_acos/dados_seirdI.txt_train-translated_divide
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
        1600  rows read from file mystery.dat                                                                                                                                                            

Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
        1600  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......    1600
Shuffling mystery data..
 Searching for best fit...
      9.652505434124     -0.000000475957                      0               1             0.0000         15444.0087             0.0001             0.0010             2.2519          1600.0000
      9.652505434015     -1.000000475957                     0>               3             1.5850         15445.5937             0.0001             0.0010             2.2519    

Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
        1600  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......    1600
Shuffling mystery data..
 Searching for best fit...
      9.652498033835      0.999999524043                      0               1             0.0000         15443.9969             0.0001             0.0010             2.2519          1600.0000
      9.652498033802     -0.000000475957                  aa~>+             280             8.1293         15452.1261             0.0001             0.0010             2.2519    

     49.879277849466   -666.000000000000               aa+a>>>*          149346            17.1883         79824.0329      12506510.5198     254193249.5796             2.6326           797.7945
Checking for brute force * 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_inverse/dados_seirdI.txt_train-translated_divide
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
        1600  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......    1600
Removing problematically
   253694028.72061795        253694.0407704

      0.164206248632     -0.999999999999                a~>~>a/           43080            15.3947           278.1247             0.0000             0.0000             1.0356           515.7953
      0.164206247666     -1.999999999999               a~>~>a/>          282354            18.1071           280.8371             0.0000             0.0000             1.0356           466.2930
      0.164206242354     -1.999999999999              aa~>~+>a/          812764            19.6325           282.3625             0.0000             0.0000             1.0356           448.8870
Checking for brute force * 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_squared/dados_seirdI.txt_train-translated_divide
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
    

In [ ]:
#R compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/","dados_seirdR.txt",60 , "7ops.txt", polyfit_deg=2, NN_epochs=200)
# Was the dynamic checked in an initial iteration? Stop the process.
# 0.000000000000+x0


Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_seirdR.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
     10.214915147212      0.000000690757                      0               1             0.0000          8171.9321             0.0001             0.0010             2.6550           800.0000
      0.000000000000      0.0000000

      5.834974090959      0.523598786465              a0>>>a>/+          742198            19.5014          4687.4807             0.0000             0.0000             2.5811            13.5574
      4.492933736207     -0.785398157094              aa~>RR>~+          765388            19.5458          3613.8928             0.0000             0.0000             2.5265            13.4586
      3.784656915976     -1.010955493307              a0>>R>R~+          765814            19.5466          3047.2722             0.0000             0.0000             2.4824            13.4669
      3.783203940270      1.010955493281              a~0>>R>R+         1394520            20.4113          3046.9745             0.0000             0.0000             2.4831            10.7610
      2.522563708724      0.574951358722              0>>>Ra>/>         1886403            20.8472          2038.8982             0.0000             0.0000             2.3263            10.0647
      2.518369353006      0.57

Checking for brute force * 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_cos/dados_seirdR.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Removing problematically
   1.0000000000000000        1.0000000474974513E-003
           0  out of          800  data points discarded for being too close to zero
Shuffling mystery data..
 Searching for best fit...
      0.143706014529      1.000000000000                     0>      

Checking polyfit 

Pareto frontier in the current branch:

Complexity #  MDL Loss #  Expression
0.0 0.0 0.000000000000+x0

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_sin/dados_seirdR.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
     10.214915146061      0.000000690757                      0               1             0.0000          8

Checking polyfit 

Pareto frontier in the current branch:

Complexity #  MDL Loss #  Expression
0.0 0.0 0.000000000000+x0

Just one variable!
Checking for symmetries...
dados_seirdR.txt_train just one variable for ADD 

dados_seirdR.txt_train just one variable for ADD 

dados_seirdR.txt_train just one variable for ADD 

dados_seirdR.txt_train just one variable for ADD 


Checking for separabilities...
dados_seirdR.txt_train just one variable for ADD
dados_seirdR.txt_train just one variable for ADD

Checking for compositionality...

Checking for generalized symmetry...



In [ ]:
#D compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/","dados_seirdD.txt",30, "7ops.txt", polyfit_deg=1, NN_epochs=600)
# Was the dynamic checked in an initial iteration? Stop the process.
# 0.000000290799+(x0/(((((0+1)+1)+1)+1)+1))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_seirdD.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
     10.296594694020      0.000000769670                      0               1             0.0000          8237.2758             0.0001             0.0010             2.6566           800.0000
     10.296491482514      0.0000007

      0.194987038671     -1.189207115003                aa*>>RR           26920            14.7164           170.7060             0.0000             0.0000             1.0409           286.3966
      0.179963599047     -1.000000000000                aa*>RRR           26992            14.7202           158.6911             0.0000             0.0000             1.0375           286.5017
      0.172641066566     -1.553773974030               aa*>>R>R          178066            17.4421           155.5549             0.0000             0.0000             1.0383           258.6174
      0.167642523167     -1.414213562373               aa*>RR>R          178138            17.4426           151.5567             0.0000             0.0000             1.0455           258.6445
      0.167642522410     -2.414213562373              aa*>RR>R>         1199224            20.1937           154.3077             0.0000             0.0000             1.0455           233.8273
      0.166052528986     -1.84

      9.480306513632      1.904129566143               a>a>>>~/          204418            17.6412          7601.8864             0.0001             0.0010             2.0963           341.9091
      9.393884759949      1.570795974563              a0>>>>>~/          770926            19.5562          7534.6640             0.0001             0.0010             2.1703           300.7515
Checking for brute force * 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_acos/dados_seirdD.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                              

      9.347051666333     -1.553773735012              aa~>>R>+R          778924            19.5711          7497.2125             0.0001             0.0010             2.1570           292.2039
      9.347051666112     -1.553773735012              aa~>>R+>R          793504            19.5979          7497.2392             0.0001             0.0010             2.1570           292.2326
      9.337874187875     -0.999999737462              a0>>R>/>R          794002            19.5988          7489.8981             0.0001             0.0010             2.1896           292.3217
Checking for brute force * 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_asin/dados_seirdD.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read f

      9.318609370703      0.223856508835               aa*Ra~>*          163726            17.3209          3558.3925             0.0000             0.0005             1.9667            83.0382
      9.187694354201     -0.540925249938              aaaa+*R~+          327504            18.3212          3509.6450             0.0000             0.0005             1.9553            80.9353
      9.178467135332      0.540360546418              aaa+*Ra~+          560824            19.0972          3506.9147             0.0000             0.0005             1.9764            78.4616
Checking polyfit 

Pareto frontier in the current branch:

Complexity #  MDL Loss #  Expression
12.0 10.15 sqrt(0.000000000001+(x0*(x0*x0)))
21.19 8.77 sqrt(-1.000000000000+sqrt(sqrt(sqrt(((x0*x0)+1)))))
72.29 7.08 sqrt(-666.000000000000*(x0+((-((x0*((x0*x0)+1))+1))+1)))
73.29 7.08 sqrt(-666.000000000000*((x0/((x0*x0)+1))+((-(x0+1))+1)))

Checking for brute force + 

Trying to solve mysteries with brute force...
Tr

      9.346988336596     -0.553773735012              aa~>>R>+R          778924            19.5711          7497.1618             0.0001             0.0010             2.1573           292.2039
      9.346988336371     -0.553773735012              aa~>>R+>R          793504            19.5979          7497.1885             0.0001             0.0010             2.1573           292.2326
      9.337929652458      0.000000262544              a0>>R>/>R          794002            19.5988          7489.9425             0.0001             0.0010             2.1895           292.3217
Checking for brute force * 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_exp/dados_seirdD.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read fr

Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
     10.296594692808      0.000000769670                      0               1             0.0000          8237.2758             0.0001             0.0010             2.6566           800.0000
     10.296491481302      0.000000769670                    aa*              16             4.0000          8241.1932             0.0001             0.0010             2.6564    

      0.172641066566     -1.553773974030               aa*>>R>R          178066            17.4421           155.5549             0.0000             0.0000             1.0383           258.6174
      0.167642523167     -1.414213562373               aa*>RR>R          178138            17.4426           151.5567             0.0000             0.0000             1.0455           258.6445
      0.167642522410     -2.414213562373              aa*>RR>R>         1199224            20.1937           154.3077             0.0000             0.0000             1.0455           233.8273
      0.166052528986     -1.847759065022              aa*>>R>>R         1203040            20.1983           153.0403             0.0000             0.0000             1.0432           234.1153
Checking for brute force * 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_squared/dados_seirdD.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of 

Checking for brute force * 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_tan/dados_seirdD.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Removing problematically
   1.0000003333334668E-003   1.0000003808309339E-006
         420  out of          800  data points discarded for being too close to zero
Shuffling mystery data..
 Searching for best fit...
     10.667393278642     -0.420235440122                      a      

In [ ]:
#Load the solution
df = pd.read_csv('SEIRD_solution.txt', delim_whitespace=True)

#Separate targets (real derivatives)
targets = [
    df['dSdt'].values,
    df['dEdt'].values,
    df['dIdt'].values,
    df['dRdt'].values,
    df['dDdt'].values
]

#Original derivative values
S = df['S'].values
E = df['E'].values
I = df['I'].values
R = df['R'].values
D = df['D'].values

feature_names = ['S', 'E', 'I', 'R', 'D']

#Predicted derivatives
dSdt_pred = -0.3 * S * I
dEdt_pred = +0.3 * S * I - 0.2 * E
dIdt_pred = -0.539112974991*(I+(I+(E/(-((E+1)+1)))))
dRdt_pred = 1.0 * I
dDdt_pred = 0.1 * I + 0.000000290799

y_preds = [dSdt_pred,dEdt_pred,dIdt_pred,dRdt_pred,dDdt_pred]

In [ ]:
res = error_eval('SEIRD', feature_names, y_preds=y_preds, targets=targets)
res

,dynamics,axis,mae,mse,r2,rmse,wilcoxon_raw_value,p-value,is_difference
0,SEIRD,S,5.152204e-22,2.279610e-41,1.000000,4.774526e-21,0.0,1.0,0
1,SEIRD,E,7.032957e-22,2.607954e-41,1.000000,5.106814e-21,0.0,1.0,0
2,SEIRD,I,1.579284e-06,1.719101e-11,0.996854,4.146205e-06,3558.0,0.0,1
3,SEIRD,R,0.000000e+00,0.000000e+00,1.000000,0.000000e+00,0.0,1.0,0
4,SEIRD,D,2.907990e-07,8.456406e-14,0.998272,2.907990e-07,0.0,0.0,1


## SIRV

In [ ]:
#Applies AIFeynman to the dynamics
# Note: Due to the supervised approach adopted in this study, each expression considers only the variables
# relevant to describing the dynamics, based on prior knowledge of the system. Thus, since each model uses
# only the variables that influence a specific target variable, it is necessary to fit a separate model
# for each one.

#S compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/","dados_sirvS.txt",300, "7ops.txt", polyfit_deg=2, NN_epochs=4000)
# Was the dynamic checked in an initial iteration? Stop the process.
# 14.61 0.0 -0.500000000000*(x0*(x1+1)) = -0.5 * S*I - 0.5* S

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_sirvS.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
      6.026745418645     -0.000000000114                      0               1             0.0000          4821.3963             0.0242             0.2375             8.3589           800.0000
      6.026745418377     -1.0000000

In [ ]:
#I compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/","dados_sirvI.txt", 300, "7ops.txt", polyfit_deg=2, NN_epochs=4000)
# Was the dynamic checked in an initial iteration? Stop the process.
# 14.61 0.0 0.000000000000+(x1*(-((x0/(-((0+1)+1)))+1)))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_sirvI.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
      7.865440277496     -0.000000004857                      0               1             0.0000          6292.3522             0.0001             0.0007             3.5877           800.0000
      7.865440272244     -1.0000000

In [ ]:
#R compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/","dados_sirvR.txt", 3600, "7ops.txt", polyfit_deg=2, NN_epochs=600)
# Was the dynamic checked in an initial iteration? Stop the process.
# 0.0 0.0 0.000000000000+x0 = 1.0*I

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_sirvR.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
      7.833871654951      0.000000004226                      0               1             0.0000          6267.0973             0.0001             0.0010             3.6709           800.0000
      0.000000000000      0.00000000

In [ ]:
#V compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/","dados_sirvV.txt", 60, "7ops.txt", polyfit_deg=2, NN_epochs=1000)
# Was the dynamic checked in an initial iteration? Stop the process.
# 4.0 0.0 0.000000000000+(x0/((0+1)+1)) = 0.5*S

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_sirvV.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
      5.867985717851      0.000000000075                      0               1             0.0000          4694.3886             0.0269             0.2495             8.2892           800.0000
      5.867881119619     -0.00000000

In [ ]:
#Load the solution
df = pd.read_csv('SIRV_solution.txt', delim_whitespace=True)

#Separate targets (real derivatives)
targets = [
    df['dSdt'].values,
    df['dIdt'].values,
    df['dRdt'].values,
    df['dVdt'].values
]

#Original derivative values
S = df['S'].values
I = df['I'].values
R = df['R'].values
V = df['V'].values

feature_names = ['S', 'I', 'R', 'V']

#Predicted derivatives
dSdt_pred = -0.5 * S*I - 0.5* S
dIdt_pred = 0.5*S*I - 1.0 * I
dRdt_pred = 1.0*I
dVdt_pred = 0.5*S

y_preds = [dSdt_pred, dIdt_pred, dRdt_pred, dVdt_pred]

In [ ]:
res = error_eval('SIRV', feature_names, y_preds=y_preds, targets=targets)
res

,dynamics,axis,mae,mse,r2,rmse,wilcoxon_raw_value,p-value,is_difference
0,SIRV,S,3.112102e-19,4.550784e-36,1.0,2.133257e-18,0.0,1.0,0
1,SIRV,I,9.401449e-22,6.015108e-41,1.0,7.755713e-21,0.0,1.0,0
2,SIRV,R,0.000000e+00,0.000000e+00,1.0,0.000000e+00,0.0,1.0,0
3,SIRV,V,9.056544e-20,9.092027e-37,1.0,9.535212e-19,0.0,1.0,0


## SIRS

In [ ]:
#Applies AIFeynman to the dynamics
# Note: Due to the supervised approach adopted in this study, each expression considers only the variables
# relevant to describing the dynamics, based on prior knowledge of the system. Thus, since each model uses
# only the variables that influence a specific target variable, it is necessary to fit a separate model
# for each one.

#S compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/","dados_sirsS.txt",3600, "7ops.txt", polyfit_deg=2, NN_epochs=600)
# Was the dynamic checked in an initial iteration? Stop the process.
# 20.25 0.0 0.100000000000*(x2+(x2+(x1*(-(x0+(x0+x0))))))



Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_sirsS.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       3
Functions used..........               +*/>~R0
 Arity            0 : 0abc
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
      9.399829827363      0.000000163819                      0               1             0.0000          7519.8639             0.0000             0.0003             3.6852           800.0000
      9.399829827301     -0.999999

      0.345675928160      0.100000002014             ccba>>~*++        21265844            24.3420           158.8100             0.0000             0.0000             1.3762             8.4843
      0.321477607889      0.199999999366             cb>R~+b>*>        78195232            26.2206           151.2754             0.0000             0.0000             1.3635             4.3359
      0.273321440885      0.100000001007            ccbaa>+~*++       170407296            27.3444           133.6665             0.0000             0.0000             1.1493             3.3165
      0.211774854626      0.200000001007            cbaaa>/+~*+       171233644            27.3514           109.7318             0.0000             0.0000             0.9310             3.3125
      0.211774854626      0.200000001007            cabba>/+~*+       171233720            27.3514           109.7318             0.0000             0.0000             0.9310             3.3125
      0.083736387270      0.19

In [ ]:
#I compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/","dados_sirsI.txt", 60, "7ops.txt", polyfit_deg=2, NN_epochs=600)
# Was the dynamic checked in an initial iteration? Stop the process.
# The results are in the "results" folder, in the file solution_dados_sirsI.txt.

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_sirsI.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
      7.798692528121     -0.000000076455                      0               1             0.0000          6238.9540             0.0001             0.0007             2.6631           800.0000
      7.798692528068     -1.0000000

     15.746470799715     -2.150099554879                aRR>RR>           43670            15.4144         12612.5910             0.0022             0.0049             5.8119            82.7657
     15.704082482113     -1.802742275947                aR>R>>R           44234            15.4329         12578.6989             0.0051             0.0108             6.5170            82.8451
     15.261285761958     -1.692928641011                aRRR>>R           44246            15.4333         12224.4619             0.0022             0.0049             6.1894            82.8699
     15.251279606728     -1.105007497037               aa>/R>RR          143274            17.1284         12218.1521             0.0017             0.0039             4.9982            74.8105
     14.482831071569     -0.535609145977               a>RR0>>/          269908            18.0421         11604.3070             0.0121             0.0248             8.3249            57.2883
     14.386712420734     -1.06

     14.387954361529     -1.069399620004              aRa>R+RRR         1450436            20.4681         11530.8315             0.0061             0.0129             7.5830            70.6139
     13.521821226230     -1.787178511516             aaa>+/>>>R         3218410            21.6179         10839.0749             0.0020             0.0044             7.4184            68.6182
     13.139170313220      0.948268139310             a>aa>R+/R~         8581270            23.0328         10534.3690             0.0018             0.0039             7.6172            64.4824
Checking for brute force * 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_acos/dados_sirsI.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       1
Functions used..........               +*/>~R0
 Arity            0 : 0a
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read fr

     14.820560857221      0.682392642902               a>a>R/>R          212158            17.6948         11874.1435             0.0124             0.0255             8.0699            49.4880
     14.820560856924      0.682392642902              aa>R~>~/R          781032            19.5750         11876.0237             0.0124             0.0255             8.0699            44.8110
     13.260626469927      0.608916982599              aaR>>+R>R          820184            19.6456         10628.1468             0.0096             0.0199             9.1247            45.3076
     13.260626469628      0.608916982599              aaR>+>R>R          871700            19.7335         10628.2346             0.0096             0.0199             9.1247            45.7181
     13.258420474364     -0.608916982634             aaR>>+R>R~         5764586            22.4588         10629.1952             0.0096             0.0199             9.1271            42.4754
     13.258420474112     -0.60

     33.627025796174     33.036012962071              a~>aa*/a/         1648878            20.6531         26922.2737            79.3811           224.1977             3.3617            45.2627
     33.336918175104   -230.822138760685             aaaRR+~>/>         2769482            21.4012         26690.9357            56.9504           156.0537             3.3615            49.9943
     33.121160130861    -28.655787223501             aa>aRR~*>/         4265202            22.0242         26518.9523            76.8486           133.6191             4.0346            49.3112
Checking polyfit 

Pareto frontier in the current branch:

Complexity #  MDL Loss #  Expression
0.0 0.19 sqrt(0.003436995408*0)
64.5 0.17 -1.399999973903*(x1/(x0+1))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_cos/dados_sirsI.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions

...

      5.220213534670      0.349979791878             aaaaa~*+**         1950498            20.8954           412.4114             0.0000             0.0002             5.4726            33.7318
      5.220213516429     -0.349979791792             aaa~*>>~*>         3024546            21.5283           413.0443             0.0000             0.0002             5.4726            31.7872
      5.220213514013     -0.349979791859             aa>~*>a~>*         5957406            22.5063           414.0223             0.0000             0.0002             5.4726            31.1026
      5.220198230197     -0.699959693256            aaaaaR~+***        13245656            23.6590           415.1739             0.0000             0.0002             5.4726            31.9976
      5.220152294624      0.349980011102            aaaaa~>****        13245816            23.6590           415.1705             0.0000             0.0002             5.4726            31.9972
      5.220121757462      0.17

      7.786418981258     -0.000000071491                aaa~>*+            7560            12.8842          6242.0194             0.0001             0.0007             2.6594           391.3376
      7.786414611289      2.999999928509                a0>a/+~           13798            13.7522          6242.8839             0.0001             0.0007             2.6594           381.6724
      7.786414610388      3.999999928509                aa>a/+~           13800            13.7524          6242.8841             0.0001             0.0007             2.6594           381.6798
      7.768133296354     -0.000000067314               aaa~>*>*           53838            15.7163          6230.2230             0.0001             0.0007             2.6707           366.5381
      7.768123824853     -1.000000067312               aa+a~>>*          149370            17.1885          6231.6876             0.0001             0.0007             2.6707           359.5256
      7.744882921996     -1.00

     52.056163574651   -666.000000000000              aaa+aa+**          458848            18.8077         29899.0455     201384663.2988    3690532346.0387             3.2335           109.3107
     52.055989904552   -666.000000000000              aa~a>RR+/          601420            19.1980         29899.3362     201384645.7830    3690532313.2659             3.2335           106.0148
     52.055981166239   -666.000000000000             aaa>RR~/>/         2603722            21.3121         29901.4453     201384644.9066    3690532311.6270             3.2335           106.0167
     52.055935132819   -666.000000000000             aa~0>>RR+/         4248370            22.0185         29902.1252     201384640.2809    3690532302.9650             3.2335           102.1038
     52.055584485163   -666.000000000000             aa~a>RRR+/         4248526            22.0185         29901.9240     201384605.2477    3690532237.2723             3.2335           102.1025
Checking polyfit 

Pareto fron

      5.220198230198     -0.699959693256            aaaaaR~+***        13245656            23.6590           415.1739             0.0000             0.0002             5.4726            31.9976
      5.220152294625      0.349980011102            aaaaa~>****        13245816            23.6590           415.1705             0.0000             0.0002             5.4726            31.9972
      5.220121757462      0.174990060366            aaaa~*>/>>/        14475448            23.7871           415.2962             0.0000             0.0002             5.4726            31.4224
Checking polyfit 

Pareto frontier in the current branch:

Complexity #  MDL Loss #  Expression
0.0 7.84 tan(-0.000031430973*0)
59.6 6.39 sqrt(-0.000001871598*((-sqrt(sqrt(sqrt(sqrt(sqrt(sqrt(sqrt(x0))))))))+1))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_squared/dados_sirsI.txt_train-translated_plus
Rejection threshold.....    10.000
Bit margin.

      7.786421180141      1.999999928509                aa*>a~/           24652            14.5894          6243.7264             0.0001             0.0007             2.6594           377.3448
      7.768139899769     -1.000000067314               aaa~>*>*           53838            15.7163          6230.2283             0.0001             0.0007             2.6707           366.5381
      7.768130427068     -2.000000067312               aa+a~>>*          149370            17.1885          6231.6929             0.0001             0.0007             2.6707           359.5256
      7.744889540790     -2.000000062658              aaa~*>>>*          420720            18.6825          6214.5941             0.0001             0.0007             2.6962           346.3802
Checking for brute force * 

Trying to solve mysteries with brute force...
Trying to solve results/mystery_world_tan/dados_sirsI.txt_train-translated_plus
Rejection threshold.....    10.000
Bit margin..............     0.000

In [ ]:
#R compartment
from aifeynman import run_aifeynman
run_aifeynman("/home/","dados_sirsR.txt", 60, "7ops.txt", polyfit_deg=2, NN_epochs=600)
# Was the dynamic checked in an initial iteration? Stop the process.
# 98.92 5.29 -0.279874964042*(x1+(-(x0+(x0+x0))))

Checking for brute force + 

Trying to solve mysteries with brute force...
Trying to solve /home/bia/AI_Feynman_scripts/dados_sirsR.txt_train
Rejection threshold.....    10.000
Bit margin..............     0.000
Number of variables.....       2
Functions used..........               +*/>~R0
 Arity            0 : 0ab
 Arity            1 : >~R
 Arity            2 : +*/
Loading mystery data....
         800  rows read from file mystery.dat                                                                                                                                                                                                                                                     
Number of examples......     800
Shuffling mystery data..
 Searching for best fit...
      9.958592717781     -0.000000263592                      0               1             0.0000          7966.8742             0.0001             0.0010             3.3151           800.0000
      8.827194581083     -0.0000001

      0.368664307558     -1.000000000000                 aa*>RR           11258            13.4587           308.3901             0.0000             0.0000             1.2013           245.8920
      0.368429373738      0.000000000000                bbbb+**           18939            14.2091           308.9526             0.0000             0.0000             1.2557           227.4508
      0.368374836668     -1.414213562373                aa*>R>R           82943            16.3398           311.0397             0.0000             0.0000             1.2125           222.2834
      0.331824153637      0.000000000000               aaba/>/*          139841            17.0934           282.5528             0.0000             0.0000             1.0074           205.9331
      0.315160753952      0.000000000000              bbbba/***          972795            19.8918           272.0204             0.0000             0.0000             1.1502           183.7260
      0.315160753952      0.00

In [ ]:
#Load the solution
df = pd.read_csv('SIRS_solution.txt', delim_whitespace=True)

#Separate targets (real derivatives)
targets = [
    df['dSdt'].values,
    df['dIdt'].values,
    df['dRdt'].values
]

#Original derivative values
S = df['S'].values
I = df['I'].values
R = df['R'].values

feature_names = ['S', 'I', 'R']

#Predicted derivatives
dSdt_pred = -1.0 * S * I - 0.2 * R
dIdt_pred =  -1.399999973903*(I/(S+1))
dRdt_pred = 0.839624892126 * I -0.279874964042 * R


y_preds = [dSdt_pred, dIdt_pred, dRdt_pred]

In [ ]:
res = error_eval('SIRS', feature_names, y_preds=y_preds, targets=targets)
res

,dynamics,axis,mae,mse,r2,rmse,wilcoxon_raw_value,p-value,is_difference
0,SIRS,S,3.890413e-05,1.451642e-08,-10.832464,1.204841e-04,75823.0,2.875936e-81,1
1,SIRS,I,7.814655e-10,2.241279e-17,1.000000,4.734214e-09,237882.0,1.757902e-01,0
2,SIRS,R,8.073495e-06,6.527090e-10,0.892399,2.554817e-05,15234.0,6.074132e-146,1
